---
title: "Manim Gallery"
subtitle: "A collection of animation previews" 
author: "Corey Voller"
date: "2025-05-15"
toc: true
toc-expand: 3
jupyter: python3
---




# Example Gallery

This gallery contains a collection of animations made using manim with video outputs and code. Feel free to copy and paste them into your projects.

## Urn Animation

<video width="640" controls>

<source src="videos/DrawUrn.mp4" type="video/mp4">

Your browser does not support the video tag. </video>


In [ ]:
#| echo: true
#| eval: false
from manim import *

class DrawUrn(Scene):
    def construct(self):
        def color_interpolate(start_color, end_color, alpha):
            start_rgb = color_to_rgb(start_color)
            end_rgb = color_to_rgb(end_color)
            interp_rgb = [(1 - alpha) * s + alpha * e for s, e in zip(start_rgb, end_rgb)]
            return rgb_to_color(interp_rgb)
        text = Text("Urn randomisation in clinical trials", font_size=32)
# Import urn svg
        urn = SVGMobject("urn.svg")
        urn.set_stroke(color="#3B2F2F", width=3)
        urn.set_fill(color="#C18F5A", opacity=1)
        urn.scale(2)
# Draw outline and fill
        text.next_to(urn, DOWN, buff=0.5)
        self.play(Write(text),run_time=2)
        self.play(DrawBorderThenFill(urn, run_time=4))
# Colour light and dark
        lighter_color = "#E6BE8A"  # Lighter color
        darker_color = "#3B2F2F"  # Darker color
        lighter_indices = [5,6,7,10]
        parts = [urn[i] for i in lighter_indices]
        anims = [
            UpdateFromAlphaFunc(part, lambda m, a: m.set_fill(color_interpolate("#C18F5A", lighter_color, a)))
            for part in parts
        ]
        dark_parts = [urn[i] for i in range(1, 5)]
        dark_anims = [
            UpdateFromAlphaFunc(part, lambda m, a: m.set_fill(color_interpolate("#C18F5A", darker_color, a)))
            for part in dark_parts
        ]
        self.play(*anims,*dark_anims, run_time=1)
        lid_parts = VGroup(urn[8], urn[10])
        rest_of_urn = VGroup(*[urn[i] for i in range(1, len(urn))])
        self.add(rest_of_urn, lid_parts)
# Move lid
# Move urn and scale
        new_text = Tex("Urn algorithm: ", font_size=32)
        ul = Underline(new_text)
        ul_text = VGroup(ul,new_text)
        ul_text.shift(UP*2 + RIGHT*2)
        self.play(urn.animate.scale(0.7).shift(LEFT * 4),
                  lid_parts.animate.scale(0.7).shift(UP * 1 + LEFT * 4).rotate(PI / 8),
                  Transform(text,ul_text),
                  run_time=2)
        self.wait(1)
        draw_text = "Draw a ball with replacement"
        # Define alpha + ball groups
        def make_alpha_ball(color):
            alpha = MathTex(r"\alpha")
            ball = Circle(radius=0.1, color=color, fill_opacity=1)
            ball.next_to(alpha, RIGHT, buff=0.2)
            return VGroup(alpha, ball), ball

        group_white, white_ball = make_alpha_ball(WHITE)
        group_red, red_ball = make_alpha_ball(RED)

        both_groups = VGroup(group_white, group_red).arrange(DOWN, aligned_edge=LEFT)
        both_groups.next_to(ul_text, DOWN, buff=1)

        self.play(Write(group_white[0]), GrowFromCenter(white_ball))
        self.wait(0.3)
        self.play(Write(group_red[0]), GrowFromCenter(red_ball))
        self.wait(0.5)

        # Move balls to starting point above scene
        white_anim = white_ball.copy()
        red_anim  = red_ball.copy()

        self.add(white_anim, red_anim )
        urn_front = urn[1]
        # Define final inside positions (center of urn)
        white_final = white_ball.copy().move_to(urn_front.get_center() + LEFT * 0.2 + DOWN * 1.2)
        red_final = red_ball.copy().move_to(urn_front.get_center() + RIGHT * 0.2 + DOWN * 1.5)
        white_final.set_z_index(-2)
        red_final.set_z_index(-2)
        white_final.set_opacity(1)
        red_final.set_opacity(1)

        # Create arcing paths
        white_path = CubicBezier(
            white_anim.get_center(),
            white_anim.get_center() + UP*2 + LEFT*1.3,
            urn_front.get_top() + UP + LEFT*0.8,
            urn_front.get_top() + DOWN * 1.5,
        )

        red_path = CubicBezier(
            red_anim.get_center(),
            red_anim.get_center() + UL * 2,
            urn_front.get_top() + UP + LEFT,
            urn_front.get_top() + DOWN * 1.5,
        )

        # Animate white ball
        self.play(MoveAlongPath(white_anim, white_path, run_time=1.8))
        self.play(white_anim.animate.scale(0.5), FadeOut(white_anim, shift=DOWN*0.2), FadeIn(white_final))
        self.wait(0.2)

        # Animate red ball
        self.play(MoveAlongPath(red_anim, red_path, run_time=1.8))
        self.play(red_anim.animate.scale(0.5), FadeOut(red_anim, shift=DOWN*0.2), FadeIn(red_final))
        self.wait(0.2)

        # Make urn transparent to reveal inside
        self.play(urn.animate.set_fill(opacity=0.3))
        # Add treatment labels
        label_white = Text("Treatment A", font_size=24).next_to(group_white, UP, buff=0.3)
        label_red = Text("Treatment B", font_size=24).next_to(group_red, UP, buff=0.3)

        # Group each alpha+ball with its label
        pair_white = VGroup(label_white, group_white)
        pair_red = VGroup(label_red, group_red)
        #side_side = VGroup(pair_white, pair_red)
       
        # Add to scene
        #self.play(side_side.animate.arrange(RIGHT, buff=1,center=True))

        # Animate down smoothly into position below ul_text
        self.play(
           pair_white.animate.move_to(ul_text.get_center() + DOWN*1.5 + LEFT * 1.5),
           pair_red.animate.move_to(ul_text.get_center() + DOWN*1.5 + RIGHT * 1.5),
            run_time=2,
            rate_func=smooth
        )
        #self.play(side_side.move_to(ul_text.get_center() + DOWN * 1.5))
        self.play(
            group_white.animate.move_to(pair_white[1].get_center()+LEFT*4),
            group_red.animate.move_to(pair_red[1].get_center()+LEFT*4),
            FadeOut(group_white[0]),
            FadeOut(group_red[0]),
            group_white[1].animate.move_to(pair_white[0].get_left()+LEFT*0.5),
            group_red[1].animate.move_to(pair_red[0].get_left()+LEFT*0.5),

          #  FadeIn(label_white, shift=UP * 0.3),
          #  FadeIn(label_red, shift=UP * 0.3),
            run_time=2,
            rate_func=smooth
        )
        self.wait(1)
        # Simulate drawing white ball
        start = white_final.get_center()
        control = start + UP * 2 + LEFT * 0.7
        end = pair_white.get_bottom() + DOWN * 0.5

# Quadratic Bezier ≈ Cubic Bezier with repeated control point
        arc_path = CubicBezier(start, control, control, end)
        # Copy the ball to animate (keep the original inside the urn)
        picked_ball = white_final.copy()
        picked_ball.set_z_index(5)
        self.add(picked_ball)

# Animate the ball moving along the arc
        self.play(MoveAlongPath(picked_ball, arc_path), run_time=2)

# Optionally, fade the picked ball into the final position (or keep it there)
        self.play(picked_ball.animate.move_to(end), run_time=0.3)
        def make_beta_ball(color):
            beta = MathTex(r"\beta")
            ball = Circle(radius=0.1, color=color, fill_opacity=1)
            ball.next_to(beta, RIGHT, buff=0.2)
            return VGroup(beta, ball)

        group_red2 = make_beta_ball(RED)
        group_red2.arrange(DOWN, aligned_edge=LEFT)
        group_red2.next_to(ul_text, DOWN*2, buff=1)

        self.play(Write(group_red2[0]), GrowFromCenter(group_red2[1]))
        self.wait(0.3)

# Path from current group position to urn
        start = group_red2.get_center()
        end = urn_front.get_center() + RIGHT * 0.1 + DOWN * 1.7

        red_path = CubicBezier(
            start,
            start + UL * 3,
            urn_front.get_top() + UP*1.1 + LEFT*1.1,
           end
        )

# Move the whole group
        self.play(MoveAlongPath(group_red2, red_path, run_time=1.8))
        self.wait(2)
        self.play(FadeOut(group_red2))
        self.play(urn.animate.set_fill(opacity=1))
        self.wait(2)

## Simple Randomisation

<video width="640" controls>

<source src="videos/SimpleRandomisation.mp4" type="video/mp4">

Your browser does not support the video tag. </video>


In [ ]:
#| echo: true
#| eval: false
from manim import *
import random

class SimpleRandomisation(MovingCameraScene):
    def construct(self):
        def make_group_box(label_text, x_shift, color):
            label = Text(label_text, color=color).to_edge(UP).shift(RIGHT * x_shift)
            slots = VGroup()
            for i in range(N):
                row, col = divmod(i, cols)
                circle = Circle(radius=radius, color=WHITE).set_fill(GREY, opacity=0.2)
                circle.move_to([x_shift + col * 0.8 - 2, 2 - row * 0.8, 0])
                slots.add(circle)
            box = SurroundingRectangle(slots, buff=0.5)
            return label, slots, box

        N = 10
        cols = 5
        radius = 0.3

        # Treatment and Control Groups (First Box)
        treat_label, treat_slots, treat_box = make_group_box("Treatment", -3.5, RED)
        ctrl_label, ctrl_slots, ctrl_box = make_group_box("Control", 3.5, BLUE)

        group1 = VGroup(treat_label, ctrl_label, treat_slots, ctrl_slots, treat_box, ctrl_box)
        sample_text1 = Text(f"Sample size: {N}", font_size=30).next_to(group1, DOWN)
        self.add(group1, sample_text1)

        # Participants
        balls = VGroup()
        for i in range(N):
            ball = Circle(radius=radius,color=WHITE).set_fill(GREY).move_to([i * 0.6 - (N * 0.6) / 2, -3.5, 0])
            balls.add(ball)
        self.add(balls)

        # Randomisation
        treat_i = ctrl_i = 0
        for i in range(N):
            choice = random.choice(["treatment", "control"])
            ball = balls[i]
            if choice == "treatment":
                target = treat_slots[treat_i]
                color = RED
                treat_i += 1
            else:
                target = ctrl_slots[ctrl_i]
                color = BLUE
                ctrl_i += 1
            self.play(ball.animate.set_fill(color).move_to(target.get_center()), run_time=0.4)
            self.play(target.animate.set_fill(color, opacity=1), run_time=0.3)

        self.wait(0.5)

        # Camera pans out and moves to new box with n = 20
        group2_origin = DOWN * 15
        new_N = 20
        new_cols = 5
        new_radius = 0.3

        def make_secondary_box(label_text, x_shift, color):
            label = Text(label_text, color=color).move_to(group2_origin + UP * 2.2 + RIGHT * x_shift)
            slots = VGroup()
            for i in range(new_N):
                row, col = divmod(i, new_cols)
                circle = Circle(radius=new_radius, color=WHITE).set_fill(GREY, opacity=0.2)
                circle.move_to(group2_origin + [x_shift + col * 0.8 - 2, 1 - row * 0.8, 0])
                slots.add(circle)
            box = SurroundingRectangle(slots, buff=0.5)
            return label, slots, box

        new_treat_label, new_treat_slots, new_treat_box = make_secondary_box("Treatment", -3.5, RED)
        new_ctrl_label, new_ctrl_slots, new_ctrl_box = make_secondary_box("Control", 3.5, BLUE)
        group2 = VGroup(new_treat_label, new_ctrl_label, new_treat_slots, new_ctrl_slots, new_treat_box, new_ctrl_box)
        sample_text2 = Text(f"Sample size: {new_N}", font_size=30).next_to(group2, DOWN)

        # Camera zoom and pan
        self.add(group2, sample_text2)
        self.play(self.camera.frame.animate.move_to(group2_origin).scale(1.5), run_time=2)
       

        # Animate randomisation again
        balls2 = VGroup()
        for i in range(new_N):
            ball = Circle(radius=new_radius,color=WHITE).set_fill(GREY).move_to(group2_origin + [i * 0.6 - (new_N * 0.6) / 2, -3.5, 0])
            balls2.add(ball)
        self.add(balls2)

        treat_i = ctrl_i = 0
        for i in range(new_N):
            choice = random.choice(["treatment", "control"])
            ball = balls2[i]
            if choice == "treatment":
                target = new_treat_slots[treat_i]
                color = RED
                treat_i += 1
            else:
                target = new_ctrl_slots[ctrl_i]
                color = BLUE
                ctrl_i += 1
            self.play(ball.animate.set_fill(color).move_to(target.get_center()), run_time=0.4)
            self.play(target.animate.set_fill(color, opacity=1), run_time=0.3)

        self.wait()

## Group sequential boundary

<video width="640" controls>

<source src="videos/BoundaryAnimation.mp4" type="video/mp4">

Your browser does not support the video tag. </video>


In [ ]:
#| echo: true
#| eval: false
from manim import *
import numpy as np


class BoundaryAnimation(Scene):
    def construct(self):
        # Define k values and boundaries (boundaries start at k=1)
        k_values = np.array([0, 1, 2, 3, 4, 5])
        a_crit = np.array([-1.61511306, -0.07126633, 0.81610852, 1.46393433, 1.986610])  # No k=0 boundary
        b_crit = np.array([4.442196, 3.141107, 2.564703, 2.221098, 1.986610])  # No k=0 boundary

        # Observed paths (starting from k=0, y=0)
        observed_red_cross = np.array([0, 1.5, 2.3, 2.7, 4.5])  # Crosses upper boundary
        observed_green_cross = np.array([0, -1.2, -1.8, -2.2, -2.5])  # Crosses lower boundary
        observed_no_cross = np.array([0,1.8, 1, 1.5, 1.8, 2.1])  # Stays within bounds

        # Create axes
        axes = Axes(
            x_range=[0, 6, 1],
            y_range=[-3, 5, 1],
            axis_config={"color": WHITE},
            x_axis_config={
                "numbers_to_include": [1, 2, 3, 4, 5],  # Only show ticks for x values from 1 to 5
            }
        )
        x_label = Tex("Analysis (k)").next_to(axes.x_axis, DOWN, buff=0.5)
        y_label = MathTex("Z_k").next_to(axes.y_axis, LEFT, buff=0.1)
        
        self.play(Create(axes), Write(x_label), Write(y_label))

        # Plot boundaries (starting at k=1)
        a_crit_line = axes.plot_line_graph(
            x_values=k_values[1:], y_values=a_crit, add_vertex_dots=False, line_color=GREEN, stroke_width=4
        )
        b_crit_line = axes.plot_line_graph(
            x_values=k_values[1:], y_values=b_crit, add_vertex_dots=False, line_color=RED, stroke_width=4
        )
        self.play(Create(a_crit_line), Create(b_crit_line))
        # Labels on graph
        reject_text = MathTex(r"\text{Reject } H_0", font_size=24)
        reject_text.move_to(axes.c2p(3, b_crit[2]) + UP * 0.5)
        self.play(Write(reject_text))

        accept_text = MathTex(r"\text{Accept } H_0", font_size=24)
        accept_text.move_to(axes.c2p(3, a_crit[2]) + DOWN * 0.4)
        self.play(Write(accept_text))

        # Function to animate each scenario
        def animate_path(observed_values):
            observed_dot = Dot(axes.c2p(k_values[0], observed_values[0]), color=BLUE)
            self.add(observed_dot)

            prev_point = axes.c2p(k_values[0], observed_values[0])
            line_segments = []
            for i in range(1, len(k_values)):
                target_point = axes.c2p(k_values[i], observed_values[i])
                new_line = Line(prev_point, target_point, color=BLUE)
                line_segments.append(new_line)

                self.play(observed_dot.animate.move_to(target_point), Create(new_line), run_time=1)

                # Check only for k >= 1 (boundaries start at k=1)
                if i >= 1:
                    print(f"k = {k_values[i]}, observed = {observed_values[i]}, a_crit = {a_crit[i-1]}, b_crit = {b_crit[i-1]}")

                    if observed_values[i] > b_crit[i-1]:  # Crossed upper boundary
                        new_line.set_color(RED)
                        self.play(observed_dot.animate.set_color(RED), new_line.animate.set_color(RED), run_time=0.5)
                        break
                    elif observed_values[i] < a_crit[i-1]:  # Crossed lower boundary
                        new_line.set_color(GREEN)
                        self.play(observed_dot.animate.set_color(GREEN), new_line.animate.set_color(GREEN), run_time=0.5)
                        break

                prev_point = target_point

            self.wait(1)
            self.play(*[FadeOut(obj) for obj in line_segments], FadeOut(observed_dot))

        # Run all three scenarios separately
        animate_path(observed_red_cross)  # Scenario 1: Turns red
        animate_path(observed_green_cross)  # Scenario 2: Turns green
        animate_path(observed_no_cross)  # Scenario 3: Stays blue

        self.wait(2)


## Multi-Arm Multi-Stage (MAMS)

<video width="640" controls>

<source src="videos/MAMS.mp4" type="video/mp4">

Your browser does not support the video tag. </video>





## 3 + 3 Dose escalation

<video width="640" controls>

<source src="videos/ThreePlusThree.mp4" type="video/mp4">

Your browser does not support the video tag. </video>


In [ ]:
#| echo: true
#| eval: false
from manim import *

class ThreePlusThree(Scene):
    def construct(self):
# Create axes
        axes = Axes(
            x_range=[0, 7, 1],
            y_range=[0, 5, 1],
            axis_config={"color": WHITE},
            x_axis_config={
                "numbers_to_include": [1, 2, 3, 4, 5],  # Only show ticks for x values from 1 to 5
            }
        )
        labels = axes.get_axis_labels(
            Tex("Time").scale(0.9), Text("Dose").scale(0.8)
        )
        self.play(Create(axes), Write(labels),run_time=1.5)
        self.wait(1)
# Define the function to add a blue arrow between cohorts
        def add_cohort_arrow(cohort_dots, y=0.2, color=BLUE, run_time=0.5):
            first_dot = cohort_dots[0]
            last_dot = cohort_dots[-1]
            start_point = first_dot.get_x()
            end_point = last_dot.get_x()
            arrow = Arrow(
                start=axes.c2p(start_point, y),
                end=axes.c2p(end_point, y),
                buff=0,
                stroke_width=6,
                tip_length=0.15,
                color=color
            )
            self.play(GrowArrow(arrow), run_time=run_time)
            self.wait(0.5)
# Draw legend
        # Green Dot
        non_dlt_dot = Dot(color=GREEN, radius=0.15)
        non_dlt_label = Text("Non-DLT").scale(0.6).next_to(non_dlt_dot, RIGHT, buff=0.1)
        # Red dot
        red_dot = Dot(color=RED, radius=0.15)
        dlt_label = Text("DLT").scale(0.6).next_to(red_dot, RIGHT, buff=0.1)
        # Dose escalation arrow
        up_arrow =  Arrow(
            start=ORIGIN,
            end=UP * 0.4,
            buff=0,
            stroke_width=5,     
            tip_length=0.15         
        )   
        esc_label = Text("Dose escalation").scale(0.6).next_to(up_arrow, RIGHT, buff=0.1)
        # Patient cohort
        cohort_square = Square(
            side_length=0.3,
            stroke_color=RED,
            stroke_width=4,
            fill_opacity=0
        )
        cohort_label = Text("Patient cohort").scale(0.6).next_to(cohort_square, RIGHT, buff=0.1)
        # Stack them vertically
        legend = VGroup(
            VGroup(non_dlt_dot, non_dlt_label),
            VGroup(red_dot, dlt_label),
            VGroup(up_arrow, esc_label),
            VGroup(cohort_square, cohort_label),
        ).arrange(DOWN, aligned_edge=LEFT, buff=0.2)
        legend.to_corner(UR, buff=0.5)
        for item in legend:
            self.play(FadeIn(item), run_time=0.5)
            self.wait(0.3)
        self.wait(3)

# Draw first dose
        positions = [0.25, 0.5, 0.75]
        dots = VGroup()
        for x in positions:
            # c2p(x, y) converts (x,y) in data coords -> scene coords
            dot = Dot(
                point=axes.c2p(x, 0.5),
                radius=0.20, 
                color=GREEN)
            self.play(FadeIn(dot), run_time=0.5)
            self.wait(0.5)
            dots.add(dot)
        box = SurroundingRectangle(
            dots,
            color=RED,
            stroke_width=4,
            buff=0.15  # space between dots and rectangle edge
        )
        self.play(Create(box), run_time=1)
        self.wait(2)
        # --- Up arrow above the third dot ---
        third_dot = dots[2]
        arrow = Arrow(
            start=third_dot.get_top() + UP * 0.05,
            end=third_dot.get_top() + UP * 1,
            buff=0,
            stroke_width=4,
            tip_length=0.15,
            color=YELLOW
        )
        self.play(GrowArrow(arrow), run_time=0.7)
        self.wait(0.5)

# Second cohort
        new_positions = [1.25, 1.5, 1.75]
        second_dots = VGroup()
        for x in new_positions:
            dot = Dot(point=axes.c2p(x, 1.5), radius=0.20, color=GREEN)
            self.play(FadeIn(dot), run_time=0.5)
            self.wait(0.2)
            second_dots.add(dot)
        second_box = SurroundingRectangle(second_dots, color=RED, stroke_width=4, buff=0.15)
        self.play(Create(second_box), run_time=1)
        # --- Up arrow above the third dot ---
        third_dot = second_dots[2]
        arrow = Arrow(
            start=third_dot.get_top() + UP * 0.05,
            end=third_dot.get_top() + UP * 1,
            buff=0,
            stroke_width=4,
            tip_length=0.15,
            color=YELLOW
        )
        self.play(GrowArrow(arrow), run_time=0.7)
        self.wait(0.5)
# Third cohort at y=2.5 between x=2 and x=3
        third_positions = [2.25, 2.5, 2.75]
        third_dots = VGroup()
        colors = [GREEN, GREEN, RED]  # Third dot is red

        for x, color in zip(third_positions, colors):
            dot = Dot(point=axes.c2p(x, 2.5), radius=0.20, color=color)
            self.play(FadeIn(dot), run_time=0.5)
            self.wait(0.2)
            third_dots.add(dot)
        third_box = SurroundingRectangle(third_dots, color=RED, stroke_width=4, buff=0.15)
        self.play(Create(third_box), run_time=1)
        self.wait(1.5)
        plus = Text("+").scale(1.2)
        plus.next_to(third_dots[-1], RIGHT, buff=0.3)
        self.play(Write(plus), run_time=0.5)
        self.wait(0.5)

# Fourth cohort at y=3.5 between x=3 and x=4
        fourth_positions = [3.55, 3.80, 4.05]
        fourth_dots = VGroup()
        colors = [GREEN, GREEN, GREEN] 
        for x, color in zip(fourth_positions, colors):
            dot = Dot(point=axes.c2p(x, 2.5), radius=0.20, color=color)
            self.play(FadeIn(dot), run_time=0.5)
            self.wait(0.2)
            fourth_dots.add(dot)
        fourth_box = SurroundingRectangle(fourth_dots, color=RED, stroke_width=4, buff=0.15)
        dashed_box = DashedVMobject(fourth_box, num_dashes=20)
        self.play(Create(dashed_box), run_time=1)
        self.wait(0.5)
        # --- Up arrow above the third dot ---
        fourth_dot = fourth_dots[2]
        arrow = Arrow(
            start=fourth_dot.get_top() + UP * 0.05,
            end=fourth_dot.get_top() + UP * 1,
            buff=0,
            stroke_width=4,
            tip_length=0.15,
            color=YELLOW
        )
        self.play(GrowArrow(arrow), run_time=0.7)
# Fifth Cohort
        fifth_positions = [4.35, 4.6, 4.85]
        fifth_dots = VGroup()
        colors = [GREEN, RED, RED] # Two red
        for x, color in zip(fifth_positions, colors):
            dot = Dot(point=axes.c2p(x, 3.5), radius=0.20, color=color)
            self.play(FadeIn(dot), run_time=0.5)
            self.wait(0.2)
            fifth_dots.add(dot)
        fifth_box = SurroundingRectangle(fifth_dots, color=RED, stroke_width=4, buff=0.15)
        self.play(Create(fifth_box), run_time=1)
        self.wait(0.5)
# End
        self.wait(1.5)